---
# Tutoriel 4 - Transfert d'apprentissage
---

<center><img src="https://python.gel.ulaval.ca/media/sio-u009/mlprocess_3.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans ce tutoriel, nous allons effectuer ce qu'on appelle le transfert d'apprentissage. Pour ce faire, nous allons utiliser un réseau à convolution de type [ResNet](https://en.wikipedia.org/wiki/Residual_neural_network) qui a été pré-entraîné avec le jeu de données [ImageNet](https://fr.wikipedia.org/wiki/ImageNet). Étant donné que les données de ImageNet sont assez semblables à ceux de CIFAR10, le réseau ResNet pré-entraîné est capable d'extraire des features (attributs) utiles à la détection d'objet. On peut donc s'attendre à de meilleurs performances sur CIFAR10. 

Pour effectuer le transfert, la mécanique est la suivante. Les poids pré-entraînés du ResNet sont chargés dans notre modèle PyTorch. Étant donné que ImageNet a 1000 classes et que CIFAR10 en a seulement 10, la couche du ResNet qui fait la classification (aussi nommé tête du réseau) est changé pour avoir seuelement 10 sorties au lieu de 1000. Une fois cela fait, nous pouvons entraîner le réseau avec CIFAR10. 

Il est possible de seulement entraîner la tête du réseau ou bien d'entraîner seulement quelques couches. Le choix du nombres de couche à entraîner va influer sur la performance du modèle ainsi que le temps d'entraînement. Entraîner la tête peut faire en sorte qu'il est possible d'extraire les représentations des exemples et donc permettre un entraînement très rapides. Au contraire, entraîner toutes les couches du réseau peut donner un modèle très performant.

In [ ]:
import math
import torch
import numpy as np
from torch import optim, nn
from torchvision import transforms
import torchvision.models as models
from torchvision.datasets.cifar import CIFAR10
from torch.utils.data import DataLoader, random_split
from torch.nn.init import kaiming_normal_, constant_

# New imports!
from poutyne.framework import Model, ModelCheckpoint, Callback, CSVLogger, EarlyStopping, ReduceLROnPlateau
from poutyne import torch_to_numpy
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Training hyperparameters
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")
batch_size = 32
learning_rate = 0.01
n_epoch = 5
num_classes = 10

In [ ]:
def load_cifar10(download=False, path='./', transform=None):
    """Loads the cifar10 dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train and test dataset
    """
    train_dataset = CIFAR10(path, train=True, download=download, transform=transform)
    test_dataset = CIFAR10(path, train=False, download=download, transform=transform)
    return train_dataset, test_dataset


def load_cifar10_with_validation_set(download=False, path='./', train_split=0.8):
    """Loads the CIFAR10 dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train, valid and test dataset ready to be ingest in a neural network
    """
    train, test = load_cifar10(download, path)
    lengths = [round(train_split*len(train)), round((1.0-train_split)*len(train))]
    train, valid = random_split(train, lengths)
    return train, valid, test

In [ ]:
norm_coefs = {}
norm_coefs['imagenet'] = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(*norm_coefs['imagenet'])
])

train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(*norm_coefs['imagenet'])
])

train, valid, test = load_cifar10_with_validation_set(download=True)

train.dataset.transform = train_transforms
valid.dataset.transform = test_transforms
test.transform = test_transforms

In [ ]:
len(train), len(valid), len(test)

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

In [ ]:
def train(name, network, params=None):
    if not params:
        params = network.parameters()
    
    optimizer = optim.SGD(params, lr=learning_rate, momentum=0.9)
    loss_function = nn.CrossEntropyLoss()
    
    early_stopping = EarlyStopping(patience=10, verbose=True)
    lr_scheduler = ReduceLROnPlateau(patience=5, verbose=True)
    callbacks = [early_stopping, lr_scheduler]

    # Poutyne Model
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Send model on GPU
    model.to(device)

    # Train
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch, callbacks=callbacks)
    return model

## Nous allons entraîner seulement la dernière couche pour classifier 10 classes.

Nous devons donc redéfinir la dernière couche du réseau ResNet-34 pour le bon nombre de classes dans notre jeu de données (10 au lieu de 1000).

In [ ]:
# Chargement du ResNet de 34 couches avec ses poids pré-entraînés sur ImageNet
net = models.resnet34(pretrained=True)

# Remplacement de la couche de classification
net.fc = nn.Linear(net.fc.in_features, num_classes)

net

In [ ]:
list(net.named_parameters())

Pour entraîner seulement la dernière couche, en PyTorch, nous pouvons seulement envoyer les paramètres de cette couche à l'optimiseur.

Les autres paramètres resteront inchangés.

Nous en profitons pour bien initialiser ces nouveaux paramètres.

In [ ]:
def get_lr_for_last_layer_only(net):
    # Filter params
    classification_layer_params = [(n, p) for n, p in net.named_parameters() if 'fc' in n]
    
    # Initialize those
    for n, p in classification_layer_params:
        if 'weight' in n:
            kaiming_normal_(p)
        if 'bias' in n:
            constant_(p, 0)
    
    # Return the list of different params/learning rates
    classification_layer_params = [p for _, p in classification_layer_params]
    return [
        {'params': classification_layer_params, 'lr': 1e-2, 'momentum':0.9},
    ]


In [ ]:
params = get_lr_for_last_layer_only(net)

In [ ]:
model = train('deep_net', net, params)

## Ici nous allons entraîner la dernière couche et peaufiner l'ensemble du réseau.

Même principe que l'étape précédente, mais nous allons spécifier différents taux d'apprentissage. Les couches pré-entraînées auront un taux d'apprentissage plus petit alors que la nouvelle couche de classification aura un taux régulier.

In [ ]:
def get_lr_for_last_layer_and_fine_tune_conv(net):
    # Filter params
    classification_layer_params = [(n, p) for n, p in net.named_parameters() if 'fc' in n]
    convolutional_layer_params = [p for n, p in net.named_parameters() if 'fc' not in n]
    
    # Initialize those
    for n, p in classification_layer_params:
        if 'weight' in n:
            kaiming_normal_(p)
        if 'bias' in n:
            constant_(p, 0)
    
    # Return the list of different params/learning rates
    classification_layer_params = [p for _, p in classification_layer_params]
    return [
        {'params': classification_layer_params, 'lr': 1e-2, 'momentum':0.9},
        {'params': convolutional_layer_params, 'lr': 1e-4, 'momentum':0.9},
    ]

In [ ]:
net = models.resnet34(pretrained=True)
net.fc = nn.Linear(net.fc.in_features, num_classes)
params = get_lr_for_last_layer_and_fine_tune_conv(net)
model = train('deep_net', net, params)

## Nous pouvons aussi effectuer un apprentissage complet du réseau et voir si les résultats s'améliorent.

In [ ]:
net = models.resnet34(pretrained=True)
net.fc = nn.Linear(net.fc.in_features, num_classes)
model = train('deep_net', net)